# Imports

In [1]:
from datetime import datetime
import firecloud.api as FAPI
import json
import os
import pandas as pd
import pprint

print(FAPI.whoami())
pp = pprint.PrettyPrinter(indent=4)

## TODO Merge in Dave Using
## TODO Read / Write from "reports" dir

pet-117272931645288568532@terra-e36fcccd.iam.gserviceaccount.com


In [2]:
BILLING_PROJECT_ID = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE = os.environ['WORKSPACE_NAME']
WORKSPACE_BUCKET = os.environ['WORKSPACE_BUCKET']

CATALOG_WORKSPACE_NAMESPACE = "anvil-datastorage"

print("Copy files:")
!gsutil cp $WORKSPACE_BUCKET/AnVILCatalogWorkspacesInput-2022-03-11.csv  .

print("List all files:")
!ls 
# print(BILLING_PROJECT_ID)

def getWorkspaceRequestorPays(bucketName):
    result = !gsutil -u anvil-jamboree-june-10-2020 requesterpays get  gs://$bucketName
    return ( "Enabled" in result[0])

def getWorkspaceBucketSize(bucketName):
    result = !gsutil -u anvil-jamboree-june-10-2020 du -s gs://$bucketName
    return result[0].split()[0]
  

# print(getBucketSize('fc-56ac46ea-efc4-4683-b6d5-6d95bed41c5e'))
    

Copy files:
Copying gs://fc-cb5be780-171f-49d4-9116-b77fd2237d0b/AnVILCatalogWorkspacesInput-2022-03-11.csv...
/ [1 files][ 58.6 KiB/ 58.6 KiB]                                                
Operation completed over 1 objects/58.6 KiB.                                     
List all files:
 1000G-high-coverage-2019_2022-03-10T07:30:15.619581.json
'Add Workspace Writer.ipynb'
 AnVILCatalogWorkspacesInput-2022-03-07.csv
 AnVILCatalogWorkspacesInput-2022-03-08.csv
 AnVILCatalogWorkspacesInput-2022-03-10.csv
 AnVILCatalogWorkspacesInput-2022-03-11.csv
 AnVILCatalogWorkspacesPublicMetadata-2022-02-25.csv
 AnVILCatalogWorkspacesPublicMetadata-2022-03-08.csv
 AnVILCatalogWorkspacesPublicMetadata-2022-03-09.csv
 AnVILCatalogWorkspacesPublicMetadata-2022-03-10.csv
 AnVILCatalogWorkspacesPublicMetadata-2022-03-11.csv
 AnVILCatalogWorkspacesPublicMetadataFromTerraAPI-2022-03-11.csv
 AnVILCatalogWorkspacesPublicMetadataFromTerraAPI-2022-03-12.csv
 AnVIL_ccdg_asc_ndd_daly_talkowski_ac-boston_asd_exo

In [3]:
## Get all workspaces we have access to
workspaces = FAPI.list_workspaces().json()


workspaceNames = set()
for w in workspaces:
    workspaceNames.add(w['workspace']['name'])
#pp.pprint(workspaces[1])
     

In [4]:


## Read in input list
input = pd.read_csv('AnVILCatalogWorkspacesInput-2022-03-11.csv')
input = input[["name"]]
input.set_index("name", inplace=True)




# print(workspaces)

## Filter out workpaces in the catalog that we have acces to
noAccess = set()
access = set()

for wName in input.index.values:
    if wName not in workspaceNames:
        noAccess.add(wName)
    else:
        access.add(wName)

print("Total catalog workspaces:" , len(access)+len(noAccess))
print("Catalog workspaces in anvil_dev:",len(access))
print("")

print("Catalog Workspaces not in anvil_dev:", len(noAccess))
print("")
print(",\n".join(noAccess))



Total catalog workspaces: 485
Catalog workspaces in anvil_dev: 485

Catalog Workspaces not in anvil_dev: 0




In [5]:

def getWorkspaceIndication(w):
    return w['workspace']['attributes']['library:indication']

def getWorkspaceDataUseRestriction(w):
    return w['workspace']['attributes']['library:dataUseRestriction']

def getWorkspaceStudyDesign(w):
    return w['workspace']['attributes']['library:studyDesign']


def getWorkspaceDataTypes(workspace):
    attributes = w["workspace"]["attributes"]
    dataTypes = attributes.get("library:datatype","Unspecified")
    if isinstance(dataTypes, dict):
        return  ",".join(dataTypes["items"])  # cant use dataType.items as items is a function on dict
    else:
        return dataTypes
    
def getWorkspaceTag(workspace, prefix):
    attributes = w["workspace"]["attributes"]
    tags = attributes.get("tag:tags","Unspecified")
    if isinstance(tags, dict):
        items = tags["items"]  # cant use dataType.items as items is a function on dict
        for tag in items:
            if tag.startswith(prefix):
                return tag.lstrip(prefix);
        return "Unspecified"
    else:
        return tags

def getWorkspaceSamples(workspace):
    return FAPI.get_entities(CATALOG_WORKSPACE_NAMESPACE, workspace,'sample').json()

def getWorkspaceSubjects(workspace):
    return FAPI.get_entities(CATALOG_WORKSPACE_NAMESPACE, workspace,'subject').json()

def getWorkspaceParticipants(workspace):
    return FAPI.get_entities(CATALOG_WORKSPACE_NAMESPACE, workspace,'participant').json()

def getWorkspaceEntityCount(entities, entity):
    e = entities.get(entity,"Unspecified")
    if isinstance(e, dict):
        return e['count']
    else:
        return 0
    
def getWorkspaceEntityCounts(workspace):
    entities = FAPI.list_entity_types(CATALOG_WORKSPACE_NAMESPACE,workspace).json()
    return {
        'discovery': getWorkspaceEntityCount(entities, "discovery"),
        'family': getWorkspaceEntityCount(entities, "family"),
        'participant': getWorkspaceEntityCount(entities, "participant"),
        'sample': getWorkspaceEntityCount(entities, "sample"),
        'subject': getWorkspaceEntityCount(entities, "subject")

    }




In [6]:
#  'accessLevel': 'READER'
reader = set()
writer = set()
for w in workspaces:
    name = w['workspace']['name']
    if name in access:
#         print(w['accessLevel'], name )
        if w['accessLevel'] == "READER":
            reader.add(name)
        if w['accessLevel'] == "WRITER" or w['accessLevel'] == "OWNER":
            writer.add(name)

print("Catalog workspaces with reader access:",len(reader))
print("")

print("Catalog Workspaces with writer access:", len(writer))
     
print("Writer workspaces")
# print(writer)

print("Reader workspaces")
# print(reader)  


Catalog workspaces with reader access: 0

Catalog Workspaces with writer access: 485
Writer workspaces
Reader workspaces


In [7]:
catalogArray = []
wsCount = 1
for w in workspaces:
    name = w['workspace']['name']   
    if name in access and name in input.index:
        print("{wsCount} - {name}".format(wsCount=wsCount, name=name))
        wsCount = wsCount+1
#         if wsCount > 3:
#             break
        entityCounts = getWorkspaceEntityCounts(name)
        bucketName = w["workspace"]["bucketName"]
        wsVals = {
            'name': name,
            'consortium': getWorkspaceTag(w,"Consortium:") ,
            'phsId': getWorkspaceTag(w,"dbGaP:") ,
            'library:dataUseRestriction':  getWorkspaceDataUseRestriction(w),
            'library:indication':  getWorkspaceIndication(w),
            'library:studyDesign': getWorkspaceStudyDesign(w),
            'library:datatype': getWorkspaceDataTypes(w),
            'subjectCount': entityCounts['subject'],
            'bucketName': bucketName,
            'sampleCount': entityCounts['sample'],
            'participantCount': entityCounts['participant'],
            'familyCount': entityCounts['family'],
            'discoveryCount': entityCounts['discovery'],
            'bucketSize': getWorkspaceBucketSize(bucketName),
            'requestorPays': getWorkspaceRequestorPays(bucketName),
            
        }
        catalogArray.append(wsVals)

catalogDF = pd.DataFrame(catalogArray)
catalogDF.set_index("name", inplace=True)
# print(catalogDF)

fileName = 'AnVILCatalogWorkspacesPublicMetadataFromTerraAPI-'+datetime.today().strftime('%Y-%m-%d')+".csv"

catalogDF.to_csv(fileName, sep="\t")
          
!gsutil cp ./$fileName $WORKSPACE_BUCKET    

1 - AnVIL_CMG_Broad_Muscle_KNC_WGS
2 - anvil_ccdg_broad_ai_ibd_daly_niddk_cho_wes
3 - AnVIL_CCDG_Broad_NP_Epilepsy_USABLC_GRU-NPU_WES
4 - AnVIL_CCDG_Broad_NP_Epilepsy_ITAUBG_DS_EPI_NPU_MDS_WES
5 - AnVIL_CMG_BaylorHopkins_HMB-NPU_WES
6 - AnVIL_CCDG_Broad_NP_Epilepsy_ITAUMC_DS_NEURO_MDS_GSA-MD
7 - ANVIL_CMG_UWASH_DS-HFA
8 - AnVIL_CCDG_Broad_NP_Epilepsy_USAMGH_MGBB_HMB_MDS_WES
9 - AnVIL_CCDG_Broad_NP_Epilepsy_USACRW_EPI_ASZ_MED_MDS_WES
10 - ANVIL_CMG_Broad_Muscle_Laing_WES
11 - AnVIL_CCDG_WashU_CVD_EOCAD_BioMe_WGS
12 - AnVIL_CCDG_Broad_NP_Epilepsy_ITAICB_HMB-NPU-MDS_GSA-MD
13 - AnVIL_CSER_CHARM_GRU
14 - AnVIL_ccdg_asc_ndd_daly_talkowski_chung_asd_exome
15 - AnVIL_CCDG_Broad_NP_Epilepsy_DEUPUM_HMB-MDS_GSA-MD
16 - AnVIL_CCDG_Broad_CVD_EOCAD_TaiChi_WGS
17 - AnVIL_CCDG_WashU_CVD_Eufam_WGS
18 - AnVIL_CCDG_Broad_AI_IBD_Brant_DS-IBD_WGS
19 - AnVIL_ccdg_asc_ndd_daly_talkowski_ac-boston_asd_exome
20 - AnVIL_CCDG_WashU_AI_T1D_T1DGC_WGS
21 - anvil_ccdg_broad_ai_ibd_daly_xavier_share_gsa
22 - AnVIL_C

173 - AnVIL_CMG_Broad_Brain_Walsh_WES
174 - anvil_ccdg_broad_ai_ibd_daly_rioux_niddk_wes
175 - AnVIL_CMG_Broad_Rare_RGP_WES
176 - 1000G-high-coverage-2019
177 - AnVIL_CCDG_Baylor_CVD_HemStroke_ERICH_WGS
178 - AnVIL_CCDG_NYGC_NP_Alz_EFIGA_WGS
179 - anvil_ccdg_broad_ai_ibd_daly_kupcinskas_wes
180 - AnVIL_CCDG_Broad_NP_Epilepsy_USACHP_GRU_GSA-MD
181 - AnVIL_CCDG_Broad_NP_Epilepsy_ITAIGI_GRU_WES
182 - AnVIL_CSER_NCGENES2_GRU
183 - AnVIL_CMH_GAFK_WGBS
184 - AnVIL_CSER_KidsCanSeq_GRU
185 - AnVIL_CCDG_Baylor_CVD_EOCAD_SoL_WGS
186 - AnVIL_CMG_Broad_Brain_NeuroDev_WES
187 - AnVIL_CCDG_Broad_CVD_AF_Roberts_UWO_WES
188 - AnVIL_CCDG_Broad_NP_Epilepsy_USAMGH_HMB_MDS_GSA-MD
189 - AnVIL_CCDG_Broad_NP_Epilepsy_USAUPN_GRU_WES
190 - AnVIL_CCDG_Broad_CVD_AF_ENGAGE_DS_WES
191 - AnVIL_CMG_Broad_Muscle_Kang_WES
192 - AnVIL_CCDG_Baylor_CVD_AFib_BioVU_WGS
193 - AnVIL_ccdg_asc_ndd_daly_talkowski_TASC_asd_exome
194 - AnVIL_CMG_Broad_Kidney_Hildebrandt_WES
195 - AnVIL_NIMH_Broad_WGSPD1_McCarroll_Pato_GRU_WGS
196

338 - anvil_ccdg_broad_daly_igsr_1kg_twist_wes
339 - AnVIL_CMG_Broad_Orphan_Lerner-Ellis_WGS
340 - AnVIL_CCDG_Broad_AI_IBD_Kugathasan_WGS
341 - anvil_ccdg_broad_ai_ibd_daly_alm_gmc_gsa
342 - AnVIL_CCDG_Broad_CVD_AF_BioVU_HMB_GSO_Arrays
343 - AnVIL_CCDG_NYGC_NP_Autism_ACE2_GRU-MDS_WGS
344 - AnVIL_CMG_Broad_Kidney_Pollak_WES
345 - anvil_ccdg_broad_ai_ibd_daly_bernstein_gsa
346 - anvil_ccdg_broad_ai_ibd_daly_silverberg_niddk_gsa
347 - AnVIL_CCDG_Baylor_CVD_HemStroke_GOCHA_DS_WGS
348 - AnVIL_CCDG_Broad_NP_Epilepsy_USACCH_DS_NEURO_MDS_GSA-MD
349 - AnVIL_CCDG_Broad_NP_Epilepsy_USACRW_DS_SEIZD_WES
350 - AnVIL_CCDG_Broad_NP_Epilepsy_USACCF_HMB-MDS_GSA-MD
351 - AnVIL_CSER_SouthSeq_GRU
352 - AnVIL_GTEx_V9_hg38
353 - AnVIL_CCDG_WashU_CVD_MultiEthnic_WGS
354 - AnVIL_CCDG_Broad_CVD_AF_Figtree_BioHeart_WES
355 - AnVIL_CMG_Yale_HMB
356 - AnVIL_CMG_Broad_Orphan_Scott_WES
357 - AnVIL_CCDG_Broad_NP_Epilepsy_USAMON_GRU_NPU_WES
358 - ANVIL_CCDG_Broad_CVD_EOCAD_PROMIS_ARRAY
359 - AnVIL_CCDG_Broad_NP_Epilep

In [72]:
pp.pprint(getEntityCounts('AnVIL_CCDG_Broad_CVD_AFib_AFLMU_WGS'))

{   'discoveryCount': 0,
    'familyCount': 0,
    'participantCount': 248,
    'sampleCount': 248,
    'subjectCount': 0}


In [66]:
pp.pprint( FAPI.list_entity_types(CATALOG_WORKSPACE_NAMESPACE, 'AnVIL_CCDG_Broad_CVD_AFib_AFLMU_WGS').json())

{   'participant': {   'attributeNames': [   'collaborator_participant_id',
                                             'gender',
                                             'participant'],
                       'count': 248,
                       'idName': 'participant_id'},
    'qc_result_sample': {   'attributeNames': [   'cram',
                                                  'freemix',
                                                  'mean_coverage',
                                                  'median_absolute_deviation',
                                                  'median_insert_size',
                                                  'pct_10x',
                                                  'pct_20x',
                                                  'pct_30x',
                                                  'pct_chimeras',
                                                  'percent_duplication',
                                                  'q20_bases